In [27]:
library(knitr)
library(moments)
library(plyr)
library(dplyr)
library(caret)
library(gridExtra)
library(scales)
library(Rmisc)
library(ggrepel)
library(xgboost)
library(glmnet)

train <- read.csv("C:/Users/Caroline/Documents/Chad Grad School/Untitled Folder/train.csv", stringsAsFactors = F)
test <- read.csv("C:/Users/Caroline/Documents/Chad Grad School/Untitled Folder/test.csv", stringsAsFactors = F)

test_labels <- test$Id
test$Id <- NULL
train$Id <- NULL

test$SalePrice <- NA
all <- rbind(train, test)
#dim(all)
summary(all$SalePrice)

#Create vectors of the numeric variables and save the names
numericVars <- which(sapply(all, is.numeric))
numericVarNames <- names(numericVars)

all_numVar <- all[, numericVars]
cor_numVar <- cor(all_numVar, use="pairwise.complete.obs") #correlations of all numeric variables

#sort on decreasing correlations with SalePrice
cor_sorted <- as.matrix(sort(cor_numVar[,'SalePrice'], decreasing = TRUE))
#select only high corelations
CorHigh <- names(which(apply(cor_sorted, 1, function(x) abs(x)>0.5)))
cor_numVar <- cor_numVar[CorHigh, CorHigh]

col_Charac <- names(all[,sapply(all, is.character)])
col_Charac

all$Foundation <- as.factor(all$Foundation)
#table(all$Foundation)

str(all$YrSold)
str(all$MoSold)
all$MoSold <- as.factor(all$MoSold)

Qualities <- c('None' = 0, 'Po' = 1, 'Fa' = 2, 'TA' = 3, 'Gd' = 4, 'Ex' = 5)
all$Heating <- as.factor(all$Heating)
#table(all$Heating)
all$HeatingQC<-as.integer(revalue(all$HeatingQC, Qualities))
all$CentralAir<-as.integer(revalue(all$CentralAir, c('N'=0, 'Y'=1)))
#table(all$CentralAir)

sum(table(all$RoofStyle))
all$RoofMatl <- as.factor(all$RoofMatl)
table(all$RoofMatl)

all$LandContour <- as.factor(all$LandContour)
table(all$LandContour)
all$LandSlope<-as.integer(revalue(all$LandSlope, c('Sev'=0, 'Mod'=1, 'Gtl'=2)))
table(all$LandSlope)

all$Exterior1st <- as.factor(all$Exterior1st)
table(all$Exterior1st)
all$Exterior2nd <- as.factor(all$Exterior2nd)
table(all$Exterior2nd)
all$MasVnrType <- as.factor(all$MasVnrType)
table(all$MasVnrType)
#all$ExterQual<- as.factor(all$ExterQual)
#table(all$ExterQual)
#all$ExterCond<- as.factor(all$ExterCond)
#table(all$ExterCond)
                             
all$BldgType <- as.factor(all$BldgType)
table(all$BldgType)
all$HouseStyle <- as.factor(all$HouseStyle)
table(all$HouseStyle)

all$Neighborhood <- as.factor(all$Neighborhood)
table(all$Neighborhood)
all$Condition1 <- as.factor(all$Condition1)
table(all$Condition1)
all$Condition2 <- as.factor(all$Condition2)
table(all$Condition2)

all$Street<-as.integer(revalue(all$Street, c('Grvl'=0, 'Pave'=1)))
table(all$Street)
all$PavedDrive<-as.integer(revalue(all$PavedDrive, c('N'=0, 'P'=1, 'Y'=2)))
#table(all$PavedDrive)

str(all$MSSubClass)
all$MSSubClass <- as.factor(all$MSSubClass)
all$MSSubClass<-revalue(all$MSSubClass, c('20'='1 story 1946+', '30'='1 story 1945-', '40'='1 story unf attic', '45'='1,5 story unf', '50'='1,5 story fin', '60'='2 story 1946+', '70'='2 story 1945-', '75'='2,5 story all ages', '80'='split/multi level', '85'='split foyer', '90'='duplex all style/age', '120'='1 story PUD 1946+', '160'='2 story PUD 1946+', '180'='PUD multilevel', '190'='2 family conversion'))

numericVars <- which(sapply(all, is.numeric)) #index vector numeric variables
factorVars <- which(sapply(all, is.factor)) #index vector factor variables
cat('There are', length(numericVars), 'numeric variables, and', length(factorVars), 'categoric variables')

all_numVar <- all[, numericVars]
cor_numVar <- cor(all_numVar, use="pairwise.complete.obs") #Correlate numeric variables

#sort on decreasing correlations with SalePrice
cor_sorted <- as.matrix(sort(cor_numVar[,'SalePrice'], decreasing = TRUE))
#select only high corelations
CorHigh <- names(which(apply(cor_sorted, 1, function(x) abs(x)>0.5)))
cor_numVar <- cor_numVar[CorHigh, CorHigh]

all$TotBathrooms <- all$FullBath + (all$HalfBath*0.5) + all$BsmtFullBath + (all$BsmtHalfBath*0.5)
#all$Remod <- ifelse(all$YearBuilt==all$YearRemodAdd, 0, 1) #0=No Remodeling, 1=Remodeling
all$Age <- as.numeric(all$YrSold)-all$YearRemodAdd
all$IsNew <- ifelse(all$YrSold==all$YearBuilt, 1, 0)
all$YrSold <- as.factor(all$YrSold)

                             
all$NeighRich[all$Neighborhood %in% c('StoneBr', 'NridgHt', 'NoRidge')] <- 4
all$NeighRich[all$Neighborhood %in% c('ClearCr', 'Crawford', 'Somerset', 'Veenker', 'Timber')] <- 3
all$NeighRich[all$Neighborhood %in% c('SawyerW', 'Gilbert', 'NWAmes', 'Blmngtn', 'CollgCr')] <- 2
all$NeighRich[all$Neighborhood %in% c('OldTown', 'Edwards', 'BrkSide', 'Sawer', 'Blueste', 'SWISU', ' NAmes', 'NPKVIII', 'Mitchel')]  <- 1
all$NeighRich[all$Neighborhood %in% c('MeadowV', 'IDOTRR', 'BrDale')] <- 0
                            
all$TotalPorchSF <- all$OpenPorchSF + all$EnclosedPorch + all$X3SsnPorch + all$ScreenPorch
all$TotBsmtFin <- all$BsmtFinSF1 + all$BsmtFinSF2

dropVars <- c('YearBuilt', 'GarageYrBlt', 'GarageArea', 'GarageCond', 'Alley','LotFrontage', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1', 'BsmtFinSF1', 'PoolQC', 'PoolArea', 'MiscFeature', 'Fence', 
              'MiscVal', 'GarageQual', 'GarageType', 'GarageFinish', 'FireplaceQu')
all <- all[,!(names(all) %in% dropVars)]
head(all$SalePrice)

#all[524,]$SalePrice

#all <- all[-524,1299] # Drop Outlier Rows

numericVarNames <- numericVarNames[!(numericVarNames %in% c('MSSubClass', 'MoSold', 'YrSold', 'SalePrice', 'OverallQual', 'OverallCond'))] #numericVarNames was created before having done anything
numericVarNames <- append(numericVarNames, c('Age', 'TotalPorchSF', 'TotBathrooms', 'TotalSqFeet'))

DFnumeric <- all[, names(all) %in% numericVarNames]

DFfactors <- all[, !(names(all) %in% numericVarNames)]
DFfactors <- DFfactors[, names(DFfactors) != 'SalePrice']
dim(DFfactors)
#typeof(DFfactors)

PreNum <- preProcess(DFnumeric, method=c("center", "scale"))
print(PreNum)
dim(DFnumeric)

DFnorm <- predict(PreNum, DFnumeric)
dim(DFnorm)

options(na.action="na.pass")
DFdummies <- as.data.frame(model.matrix(~.-1, DFfactors))
dim(DFdummies)

ZerocolTest <- which(colSums(DFdummies[(nrow(all[!is.na(all$SalePrice),])+1):nrow(all),])==0)
colnames(DFdummies[ZerocolTest])
DFdummies <- DFdummies[,-ZerocolTest]
dim(DFdummies)

#check for absent values
ZerocolTrain <- which(colSums(DFdummies[1:nrow(all[!is.na(all$SalePrice),]),])==0)
ZerocolTrain
colnames(DFdummies[ZerocolTrain])
DFdummies <- DFdummies[,-ZerocolTrain]
dim(DFdummies)

fewOnes <- which(colSums(DFdummies[1:nrow(all[!is.na(all$SalePrice),]),])<10)
fewOnes
colnames(DFdummies[fewOnes])
DFdummies <- DFdummies[,-fewOnes] #removing predictors
dim(DFnorm)
dim(DFdummies)


dim(DFdummies)
combined <- cbind(DFnorm, DFdummies) #combining all into one Data Frame

dim(combined)

train1 <- combined[!is.na(all$SalePrice),]
dim(train1)
#train1<-train1[-524,]
test1 <- combined[is.na(all$SalePrice),]
dim(test1)

xgb_grid = expand.grid(
  nrounds = 1000,
  eta = c(0.05),
  max_depth = c(2, 3),
  gamma = 0,
  colsample_bytree=1,
  min_child_weight=c(1, 2, 3),
  subsample=1
)
my_control <-trainControl(method="cv", number=5)

#Takes a long time to run this care to find the best hyperbarameter
#xgb_caret <- train(x=train1, y=all$SalePrice[!is.na(all$SalePrice)], method='xgbTree', trControl= my_control, tuneGrid=xgb_grid) 
#xgb_caret$bestTune

label_train <- all$SalePrice[!is.na(all$SalePrice)]

# put our testing & training data into two seperates Dmatrixs objects
dtrain <- xgb.DMatrix(data = as.matrix(train1), label= label_train)
dtest <- xgb.DMatrix(data = as.matrix(test1))
summary(dtrain)
default_param<-list(
  objective = "reg:squarederror",
  booster = "gbtree",
  eta=0.05, #default = 0.3
  gamma=0,
  max_depth=3, #default=6
  min_child_weight=4, #default=1
  subsample=1,
  colsample_bytree=1
)

#Cross validate to see the ideal number of rounds
xgbcv <- xgb.cv(params = default_param, data = dtrain, nrounds = 500, nfold = 5, showsd = T, stratified = T, print_every_n = 40, early_stopping_rounds = 10, maximize = F)

xgb_mod <- xgb.train(data = dtrain, params=default_param, nrounds = xgbcv$best_iteration)

XGBpred <- predict(xgb_mod, dtest)
XGBpred

SalePrice<-XGBpred

Id<-c(951:1450)
mtx<-cbind(Id, SalePrice)
mtx

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max.    NA's 
  35311  129900  164000  180599  214000  745000     500 

[1] "MSZoning"      "Street"        "Alley"         "LotShape"     
 [5] "LandContour"   "Utilities"     "LotConfig"     "LandSlope"    
 [9] "Neighborhood"  "Condition1"    "Condition2"    "BldgType"     
[13] "HouseStyle"    "RoofStyle"     "RoofMatl"      "Exterior1st"  
[17] "Exterior2nd"   "MasVnrType"    "ExterQual"     "ExterCond"    
[21] "Foundation"    "BsmtQual"      "BsmtCond"      "BsmtExposure" 
[25] "BsmtFinType1"  "BsmtFinType2"  "Heating"       "HeatingQC"    
[29] "CentralAir"    "Electrical"    "KitchenQual"   "Functional"   
[33] "FireplaceQu"   "GarageType"    "GarageFinish"  "GarageQual"   
[37] "GarageCond"    "PavedDrive"    "PoolQC"        "Fence"        
[41] "MiscFeature"   "SaleType"      "SaleCondition"

 int [1:1449] 2009 2009 2006 2008 2008 2008 2006 2006 2007 2009 ...
 int [1:1449] 8 6 4 3 4 5 1 11 8 5 ...


The following `from` values were not present in `x`: None


[1] 1449

...

[1] 500 164

     Length       Class        Mode 
          1 xgb.DMatrix externalptr 

[1]	train-rmse:186385.562500+1385.689321	test-rmse:186375.478125+5673.916561 
Multiple eval metrics are present. Will use test_rmse for early stopping.
Will train until test_rmse hasn't improved in 10 rounds.

[41]	train-rmse:35695.430469+698.229456	test-rmse:40161.373437+4360.160109 
[81]	train-rmse:19290.182422+695.628125	test-rmse:29323.374609+6120.492917 
[121]	train-rmse:16060.180273+641.983593	test-rmse:28375.422266+7392.554956 
[161]	train-rmse:14486.499414+592.391221	test-rmse:28237.025391+7840.105124 
Stopping. Best iteration:
[177]	train-rmse:14019.815039+575.416261	test-rmse:28178.858203+7907.675432



[1]  97111.91 192393.39 344995.69 586675.38 101063.46 166247.52 202021.91
  [8] 238780.20 147843.81 120618.49 136035.80 150863.88 166971.67 170459.92
 [15] 158656.77 108511.60 192991.53 191901.81 165646.36 145071.09 170306.03
 [22]  71125.49 308726.06  89942.90 163109.70 151861.44 311796.78 134799.30
 [29] 184281.38 310519.56 152066.38  91962.24 127018.24 170420.31 391059.31
 [36] 291313.84 145282.97 227330.41 175438.80 162186.41 171959.67 379920.69
 [43] 102920.02 110943.68 139523.77  82459.17 194177.34 133121.98 136560.34
 [50] 188633.83 148954.58 415402.50 147792.22 139573.70 316424.84 259620.55
 [57] 190061.97 155559.14 117717.59 371821.88 150950.20 143484.95 141666.94
 [64] 142496.20 119748.97  94945.34 124080.27 178574.34 347442.16 136854.94
 [71]  97787.52 126546.65 280045.53 118210.99  92711.38  99350.67 117057.38
 [78] 218440.22 306055.81 136504.91 120794.84 168126.28 293681.97 145610.06
 [85] 266861.91 195153.20 129473.21  90035.55 273010.69 301685.47 212799.61
 [92] 215851.06 181740.31 107889.92  60780.64 135476.69 289474.44 126060.37
 [99]  76650.48 164537.52 235150.91 288886.66 126826.98 253584.95 371559.88
[106] 139623.08 187270.08 150184.84 304958.81 230583.55 164241.58  73872.00
[113] 140486.20 144201.86 161450.59  92705.16 130256.69 131057.09 258594.62
[120] 274979.41 136574.56 119323.32  95181.06 149457.08  96968.27 140974.97
[127] 188415.08 212937.38 110259.31 181399.98 235344.59 110817.84 112036.90
[134]  79143.24 248710.38 125139.50 132336.64 137104.06 218335.16 281031.12
[141] 179115.20 124785.15 116788.41 121754.48 182401.11 191938.00 157174.75
[148] 285424.31 169152.27 168958.94 140315.34 127618.61 171340.30 149587.34
[155] 241368.59 183735.08 155986.83 188299.31 138134.31 190219.05 117328.20
[162] 157494.77 196269.02 295456.81 122974.42 112217.88 173542.88 296057.78
[169] 139600.33 157379.88 169351.33 195475.33 124618.83 103195.14  91090.81
[176] 143195.14 186476.31 405075.56 285391.38 291561.97 139883.55 110176.90
[183] 129513.06 126467.59 237242.59 182088.17 107618.86 175930.77 121742.51
[190] 134420.38  94185.24 111088.09 237012.42 310174.88 264036.69 123477.88
[197] 172680.17 250022.25 181571.59 167568.05 224172.44 119072.27 301352.06
[204] 156708.17 140011.09 190254.47 110000.52 131734.80 155068.00 212981.94
[211] 146680.72 145061.78  92934.93 249921.22 206856.16 117676.29 202698.30
[218] 291776.41 111690.77 147690.78 191462.58 175621.03 247815.11 144079.28
[225] 145184.77 131000.52 278037.75 143426.12 201551.81 177386.88 194388.70
[232] 197186.84 213886.19 121465.28 118547.90 218126.86 167879.25 152291.31
[239] 324033.81 108759.12 151370.42 113415.06 125636.27 126335.47 205344.67
[246] 249795.03 111950.06 111098.52 250705.67 249178.11 190187.77 185138.73
[253] 170686.58 126332.23 231106.31 232198.23 250171.95 198785.53 238825.73
[260] 181143.72 132998.58 155023.02 135473.31 133905.89 189067.00 126540.97
[267] 229681.50 174096.16 212335.12 144400.14 124724.62 116131.60 122888.20
[274] 124368.60 188521.77 187680.36 139270.64 231444.52 191933.02 191122.72
[281] 234825.56 159376.23 412840.56 164939.06 158902.78 199264.64 101723.45
[288] 178213.34 102905.80  96629.41 351899.16 136528.69 324182.34 215070.61
[295] 155428.38 155030.92 176887.25 256218.41 385351.81 147061.45 129099.95
[302] 105195.13 268432.53 154910.00 140845.23 226844.77 174571.41 157543.27
[309] 159152.00 160695.03 195007.22  87246.53 223261.50 197870.75 101832.36
[316] 118419.62 171900.00 295849.75 307828.38 147243.12 289158.91 122264.74
[323] 230226.19 367085.31 267773.88 112891.14 167905.59  95307.81 166874.11
[330] 176794.70 127003.96 127685.38 359269.91 311024.69 207448.19 137051.67
[337] 164380.97 190745.27 225859.22 175541.08 221772.73 214386.05 170082.75
[344] 283377.56 247597.05 141666.94 187226.89 273541.94 149375.73  72284.15
[351] 114990.03 156400.34 225220.20 129957.62 174180.31 221270.58 139143.56
[358] 264179.09 122181.92 149053.34 174303.50 178302.66 124809.23 130562.33
[365] 105567.13 299285.47 191827.70 121137.68 2373

Id,SalePrice
951,97111.91
952,192393.39
953,344995.69
954,586675.38
955,101063.46
956,166247.52
957,202021.91
958,238780.20
959,147843.81
960,120618.49


In [28]:
write.csv(mtx,file="C:/Users/Caroline/Documents/Chad Grad School/Untitled Folder/Prices6.csv",row.names=F)
                             